In [ ]:
# default_exp deepzoom

# Converting to deep zoom package 

Converting an image into a deep zoom pyramid and publishing as deep zoom package in a b2 bucket, is a three step process: 1) convert to dzi pyramid, 2) upload pyramid to bucket, 3) create html and jsonp viewers.   

In [ ]:
cd /home/frank/Work/Werkmappen_Software/dzi2b2 

/mnt/datadisk/Work/Werkmappen_Software/dzi2b2


In [ ]:
#hide
fname1 = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1930-22_highres.png' 
fname2 = '/home/frank/Work/Projecten/DoRe/data/drawings/rma-web-highres/RP-T-1930-51_highres.png'

In [ ]:
#hide
import myb2keys

application_key = myb2keys.application_key 
application_key_id = myb2keys.application_key_id 
bucket_name = myb2keys.bucket_name 

In [ ]:
#export 
import pyvips 
import os 
import shutil 
import glob
import re 
from bs4 import BeautifulSoup 
import sys
import time 
import b2sdk.v1 as b2
import numpy as np
import json 
import requests 

ROOTDIR = 'deepzoom'
VIEWERSDIR = 'viewers'

class DeepZoomStore: 
    
    def __init__(self): 
        '''Create local deep zoom store directory for one or more images. '''
        
        store_dir = os.path.join('.', ROOTDIR)
        os.makedirs(store_dir, exist_ok=True) 
        
        viewers_dir = os.path.join('.', ROOTDIR, VIEWERSDIR)
        os.makedirs(viewers_dir, exist_ok=True) 
        
        self.images = []

        
    def dzp_save(self, fname): 
        
        name = dzp_save(fname) 
        self.images.append(name)
        
        
    def connect_b2_bucket(self, application_key_id, application_key, bucket_name): 
        
        self.application_key_id = application_key_id 
        self.application_key = application_key
        self.bucket_name = bucket_name 
        
        self.b2_api, self.base_url = connect_b2_bucket(application_key_id, application_key, bucket_name)
        
        
    def make_webviewers(self): 
        
        for name in self.images: 
        
            dzi_to_js(name, self.base_url, self.bucket_name)
            make_html(name, self.bucket_name) # single image viewer 


    def upload(self):
        
        upload(self.b2_api, self.bucket_name)
        
        
    def create_multiviewer(self, viewer_name, selection='all'): 
        
        print('Automatically updating b2 bucket to include all newly created images...') 
        self.upload()
        
        create_multiviewer(viewer_name, self.application_key_id, self.application_key, 
                           self.bucket_name, selection=selection)
        
        print(f'Automatically uploading multiviewer: {viewer_name}...')
        
        self.upload()
        
        
        
        
# Functions                   

def upload(b2_api, bucket_name):
    ''''''

    source = os.path.join('.', ROOTDIR)
    destination = f'b2://{bucket_name}/{ROOTDIR}'  

    source = b2.parse_sync_folder(source, b2_api)
    destination = b2.parse_sync_folder(destination, b2_api)

    policies_manager = b2.ScanPoliciesManager(exclude_all_symlinks=True)

    synchronizer = b2.Synchronizer(
            max_workers=10,
            policies_manager=policies_manager,
            dry_run=False,
            allow_empty_source=True,
        )

    no_progress = False 

    # need to run this to actually synchronize 

    with b2.SyncReport(sys.stdout, no_progress) as reporter:
            synchronizer.sync_folders(
                source_folder=source,
                dest_folder=destination,
                now_millis=int(round(time.time() * 1000)),
                reporter=reporter,
            ) 
            
        
    

def make_html(name, bucket_name): 
    
    html_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}_view.html') 
    
    html = f'''
<!DOCTYPE html>
<html>
<body style="width: 800px; height: 600px; background-color: snow">

<div id="openseadragon1" style="width: 100%; height: 100%; background-color: gray"></div> 

<script src="https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/openseadragon.min.js" 
        integrity="sha512-qvQYH6mPuE46uFcWLI8BdGaJpB5taX4lltbSIw5GF4iODh2xIgyz5ii1WpuzPFUknHCps0mi4mFGR44bjdZlZg==" 
        crossorigin="anonymous">
</script>

<script type="text/javascript">
var viewer = OpenSeadragon({{
    id: "openseadragon1",
    prefixUrl: "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
    tileSources: "{name}.dzi", 
 }});
</script>


</body>
</html>  
'''
    with open(html_fname, 'w') as fh: 
        
        print(f'Saving: {html_fname}')
        fh.writelines(html)
       


    
    

def connect_b2_bucket(application_key_id, application_key, bucket_name): 
    
    '''Creates upload connection to your Backblaze b2 bucket.'''
    
    info = b2.InMemoryAccountInfo()
    b2_api = b2.B2Api(info) 
    b2_api.authorize_account("production", application_key_id, application_key)  
    
    
    bucket = b2_api.get_bucket_by_name(bucket_name) 

    filenames = [file_info.file_name for file_info, _ in bucket.ls(show_versions=False, recursive=False)]
    urls = [b2_api.get_download_url_for_file_name(bucket_name, f) for f in filenames] 
    base_url = re.split(bucket_name, urls[0])[0] 
    
    return b2_api, base_url


def dzp_save(fname): 
    # name = dzp_save(fname, report_tree=report_tree) 
    
    name, ext = os.path.splitext(os.path.basename(fname))
    dzp_fname = os.path.join('.', ROOTDIR, f'dzp_{name}')
    dzi_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.dzi') 
    dzi_base = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}')
    
    os.makedirs(dzp_fname, exist_ok=True) 
    
    if not os.path.exists(dzi_fname): 
        print('hi')
        v_img = pyvips.Image.new_from_file(fname) 
        v_img.dzsave(dzi_base)
    else: 
        print(f'Not overwriting existing files: {dzi_base}')
    
    return name 


    
    
def dzi_to_js(name, base_url, bucket_name): 
    '''Parse XML.dzi file into JSONP.js file. 
    
    '''
    
    url = f'{base_url}{bucket_name}/{ROOTDIR}/dzp_{name}/{name}_files/'  
    dzi_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.dzi') 
    jsonp_fname = os.path.join('.', ROOTDIR, f'dzp_{name}', f'{name}.js') 
    
    with open(dzi_fname, 'r') as fh: 
        lines = fh.read()
         
    soup = BeautifulSoup(lines, 'xml')

    Format = soup.Image.attrs['Format']
    Overlap = soup.Image.attrs['Overlap']
    TileSize = soup.Image.attrs['TileSize']
    Height = soup.Image.Size.attrs['Height'] 
    Width = soup.Image.Size.attrs['Width']

    jsonp = f'''OpenSeadragon( \
    {{
    "id": "{name}", 
    "prefixUrl": "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
    "tileSources": {{
        "Image": {{
           "xmlns":    "http://schemas.microsoft.com/deepzoom/2008",
           "Url":      "{url}", 
           "Format": "{Format}", 
           "Overlap": "{Overlap}",  
           "TileSize": "{TileSize}", 
           "Size": {{
               "Height": "{Height}", 
               "Width": "{Width}" 
            }}
        }}
    }}
    }}); 
    '''
    
    with open(jsonp_fname, 'w') as fh: 
        print(f'Saving: {jsonp_fname}')
        fh.writelines(jsonp)



def create_multiviewer(viewer_name, application_key_id, application_key, bucket_name, selection='all'): 
    '''Download all single image JSONP.js files and combine in multi-viewer.js '''
    
    # not yet implemented selection possibility 
    
    # connect 
    b2_api, base_url =connect_b2_bucket(application_key_id, application_key, bucket_name)
    bucket = b2_api.get_bucket_by_name(bucket_name) 
    
    # list all files in deepzoom folder 
    filenames = [file_info.file_name for file_info, _ in 
                 bucket.ls(folder_to_list=ROOTDIR, 
                           show_versions=False, recursive=True)]
    
    # select (single image) /dzp_<name>/<name>.js jsonp files 
    is_js = [f.endswith('.js') and f.startswith('deepzoom/dzp_') for f in filenames]
    js_fnames = np.array(filenames)[is_js]
    
    # construct their urls
    js_urls = [f'{base_url}{bucket_name}/{j}' for j in js_fnames] 
    
    # download and parse .js files and append into multi tileSources string 
    jsonp_strings = []

    ptrn = '.*"tileSources":(?P<tsources>.*)\s*\}\s*\);' 
    repl = '\g<tsources>'

    for u in js_urls: 
        # download jsonp.js file 
        s = requests.get(u).content.decode() 
        # extract tileSources substring 
        s = re.sub(ptrn, repl, s, flags=re.DOTALL)
        # check valid json and improve formatting
        s = json.dumps(json.loads(s), indent=2)

        jsonp_strings.append(s)

    multi_tileSources = ',\n'.join(jsonp_strings)
    
    
    # add openseadragon callback and viewer settings 
    multi_jsonp = f'''OpenSeadragon( \
{{
"id": "{viewer_name}", 
"prefixUrl": "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
"tileSources": [{multi_tileSources}], 
"sequenceMode": "true", 
"preserveViewport": "false", 
"showReferenceStrip": "true"
}}); 
'''
    
    # write to local multiviewer.js file  
    multiviewer_fname = os.path.join('.', ROOTDIR, VIEWERSDIR, f'{viewer_name}.js')
    
    with open(multiviewer_fname, 'w') as fh: 
        print(f'Saving: {multiviewer_fname}')
        fh.writelines(multi_jsonp)
   

In [ ]:
#hide 
dz = DeepZoomStore()
dz.dzp_save(fname1)

dz.connect_b2_bucket(application_key_id, application_key, bucket_name)
dz.make_webviewers()

Not overwriting existing files: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres
Saving: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.js
Saving: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_view.html


In [ ]:
dz.create_multiviewer('all_test3')

Automatically updating b2 bucket to include all newly created images...
upload dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.js            
upload viewers/all_test2.js                                              
upload dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_view.html           
Saving: ./deepzoom/viewers/all_test3.js                                  
Automatically uploading multiviewer: all_test3...
upload viewers/all_test3.js                                           


In [ ]:
dz.upload()

Not overwriting existing files: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres
Saving: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.js
Saving: ./deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_view.html
upload dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.js            
upload dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_view.html           
upload viewers/all_test.js                                               


## Testing

For a start, I would like to generate a viewer for all deep zoom images. We can think next about selections...  

In [ ]:
b2_api, base_url =connect_b2_bucket(application_key_id, application_key, bucket_name)
#def deepzoom_list(b2_api): 
bucket = b2_api.get_bucket_by_name(bucket_name) 

In [ ]:
base_url

'https://f002.backblazeb2.com/file/'

In [ ]:
viewer_name = 'all_test2'

create_multiviewer(viewer_name, application_key_id, application_key, bucket_name, selection='all')

Saving: ./deepzoom/viewers/all_test2.js


Where are my .js files? 

In [ ]:
base_url

'https://f002.backblazeb2.com/file/'

In [ ]:
filenames = [file_info.file_name for file_info, _ in bucket.ls(folder_to_list=ROOTDIR, show_versions=False, recursive=True)]

In [ ]:
filenames

['deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.dzi',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.js',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/0/0_0.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/1/0_0.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/0_0.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/0_1.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/0_2.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/1_0.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/1_1.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/1_2.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/2_0.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/2_1.jpeg',
 'deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres_files/10/2_2.jpeg',
 'deepzoom/dzp_RP-T-1930-22_hi

In [ ]:
is_js = [f.endswith('.js') and f.startswith('deepzoom/dzp_') for f in filenames]

In [ ]:
js_fnames = np.array(filenames)[is_js]

js_urls = [f'{base_url}{bucket_name}/{j}' for j in js_fnames]



js_urls
    

['https://f002.backblazeb2.com/file/dore-data/deepzoom/dzp_RP-T-1930-22_highres/RP-T-1930-22_highres.js',
 'https://f002.backblazeb2.com/file/dore-data/deepzoom/dzp_RP-T-1930-51_highres/RP-T-1930-51_highres.js']

In [ ]:
jsonp_strings = []

ptrn = '.*"tileSources":(?P<tsources>.*)\s*\}\s*\);' 
repl = '\g<tsources>'

for u in js_urls: 
    # download jsonp.js file 
    s = requests.get(u).content.decode() 
    # extract tileSources substring 
    s = re.sub(ptrn, repl, s, flags=re.DOTALL)
    # check valid json and improve formatting
    s = json.dumps(json.loads(s), indent=2)
     
    jsonp_strings.append(s)
    
#jsonp_strings = np.array(jsonp_strings) 

multi_tileSources = ',\n'.join(jsonp_strings)

In [ ]:


multi_jsonp = f'''OpenSeadragon( \
{{
"id": "{viewer_name}", 
"prefixUrl": "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/2.4.2/images/", 
"tileSources": [{multi_tileSources}], 
"sequenceMode": "true", 
"preserveViewport": "false", 
"showReferenceStrip": "true"
}}); 
'''

In [ ]:
#print(multi_jsonp)

In [ ]:
multiviewer_fname = os.path.join('.', ROOTDIR, VIEWERSDIR, f'{viewer_name}.js')

In [ ]:
with open(multiviewer_fname, 'w') as fh: 
    print(f'Saving: {multiviewer_fname}')
    fh.writelines(multi_jsonp)

Saving: ./deepzoom/viewers/all_test.js


Works! 